In [1]:
# Set up a basic configuration.  Change as needed.
import dask.distributed
import plotly.express as px
from dask import delayed
import large_image
import numpy as np

runTimeInfo = []
dask.distributed.Client(n_workers=2)


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 12,Total memory: 30.99 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36413,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 30.99 GiB
Comm: tcp://127.0.0.1:46747,Total threads: 6
Dashboard: http://127.0.0.1:36249/status,Memory: 15.49 GiB
Nanny: tcp://127.0.0.1:43945,


In [ ]:
# !pip install fast_histogram

In [2]:
import time
@dask.delayed
def computeTileHist( slide_path, position, count, kwargs ):
    ### Count is the number of tiles to process in a single task
    
    ts = large_image.getTileSource(slide_path)
    
    results = []
    for pos in range(position, position+count):
        st = time.time()
        
        tile_info = ts.getSingleTile(tile_position=pos,**kwargs)
        image_array = tile_info['tile']
        avgColor  = np.mean(image_array, axis=(0,  1))

        
        histData = {'topX':tile_info['gx'],'topY':tile_info['gy'],'width':tile_info['gwidth'],
                    'height':tile_info['gheight'], 'bytes_read': image_array.size,
                   "apiURL": dsaApiUrl,   "imageId": imageInfo['imageId'],    "imageName": imageInfo['imageName'],
                   "average": list(avgColor), "featureType": "imgHistogram", "startTime": st ,
                    "genAvgColor": time.time() -st}

        st = time.time()
        for idx,c in enumerate(['red','green','blue']):
            h = np.histogram(image_array[:,:,idx],bins=255,range=[0,255])
            histData[c] =h[0].tolist()
            
        histData['computeHist'] = time.time() - st
        
        results.append(histData) 

    return results

In [15]:
%%time

imageInfo = {}
imageInfo['imageId'] = 2
imageInfo['imageName'] = 'TBD'
tile_grouping = 32

slide_path = './TCGA-3L-AA1B-01A-01-TS1.9C415218-D5B4-4945-B243-F42A4C8C0484.svs'


import glob
for slide_path in glob.glob("SmallSampleFiles/*.svs"):

    for tileSize in [1000,2000,4000,8000,12000,16000,32000]:


        region = dict(
            width=tileSize, height=tileSize,
        )

        ts = large_image.getTileSource(slide_path)
        dsaApiUrl = "TBD"
        kwargs = dict(format=large_image.tilesource.TILE_FORMAT_NUMPY,region=region)

        im_region = ts.getRegion(region=region, format=large_image.tilesource.TILE_FORMAT_NUMPY)[0]
        total_tiles = ts.getSingleTile(**kwargs)['iterator_range']['position']


        st = time.time()
        results = []
        for position in range(0, total_tiles, tile_grouping):
            results.append(computeTileHist(slide_path, position, min(tile_grouping, total_tiles - position), kwargs))
        results = dask.compute(results)


        et = time.time() - st
        print(et,"for tilesize of",tileSize)
        runTimeInfo.append({'runTime': et, "region": region,
                            "tileSize": tileSize, "slide_path": slide_path,"tile_grouping": tile_grouping})

0.4310011863708496 for tilesize of 1000
0.6991410255432129 for tilesize of 2000
1.7476589679718018 for tilesize of 4000
5.135886192321777 for tilesize of 8000
3.847784996032715 for tilesize of 12000
4.397916555404663 for tilesize of 16000
4.066150903701782 for tilesize of 32000
0.2333827018737793 for tilesize of 1000
0.3925912380218506 for tilesize of 2000
1.5224485397338867 for tilesize of 4000
4.453020811080933 for tilesize of 8000
4.461623668670654 for tilesize of 12000
4.337134838104248 for tilesize of 16000
4.217117786407471 for tilesize of 32000
0.4633502960205078 for tilesize of 1000
0.6242806911468506 for tilesize of 2000
1.7247600555419922 for tilesize of 4000
6.655493259429932 for tilesize of 8000
7.780604839324951 for tilesize of 12000
7.248055458068848 for tilesize of 16000
7.621370077133179 for tilesize of 32000
0.542773962020874 for tilesize of 1000
0.8384852409362793 for tilesize of 2000
1.4832570552825928 for tilesize of 4000
3.0840797424316406 for tilesize of 8000
4.38

In [16]:
import pandas as pd


df = pd.DataFrame(runTimeInfo)

px.scatter(df,x='runTime', y='tileSize',title="Run time vs tileSize with n_workers=2")

['SmallSampleFiles/TCGA-19-1787-01C-01-TS1.b9f6f0f2-14f2-4bc5-b7f8-9520ec38eb98.svs',
 'SmallSampleFiles/TCGA-08-0509-01A-01-TS1.27f6ae4c-e445-4e2e-a94b-31d449ba69c9.svs',
 'SmallSampleFiles/TCGA-12-0654-01C-01-BS1.52cba7d5-7130-48c1-a31e-349f8b12822b.svs',
 'SmallSampleFiles/TCGA-06-5417-01A-01-TS1.a84b8f46-a57d-469d-a65e-4aa5f45da5f3.svs',
 'SmallSampleFiles/TCGA-19-1387-01A-01-TS1.ff52185b-1cd8-49d4-bb3d-739b5b75b6fb.svs',
 'SmallSampleFiles/TCGA-06-5412-01A-01-TS1.4e47da5e-588a-4cc1-8c10-a701764b1f7c.svs',
 'SmallSampleFiles/TCGA-28-5211-01D-01-TS1.c05a78f8-ca45-4493-b96f-ab70fadd3a2c.svs',
 'SmallSampleFiles/TCGA-19-2623-01B-01-TS1.f7894961-4e2c-4bfd-90df-442d1933fef9.svs',
 'SmallSampleFiles/TCGA-12-0820-01Z-00-DX1.43DEB9E2-D419-4263-8188-20FEBB070AAF.svs',
 'SmallSampleFiles/TCGA-12-0620-01Z-00-DX1.5d4cc50f-9b78-4b8b-8e8e-1c20ae415d71.svs',
 'SmallSampleFiles/TCGA-06-5416-01A-01-TS1.1749a247-c33d-4a06-88f2-4deef3c5d982.svs',
 'SmallSampleFiles/TCGA-06-5414-01A-01-TS1.660d19f8-12

In [ ]:
df

In [18]:
flattened_results = []

for ta in results:
    for tb in ta:
        for tc in tb:
            flattened_results.append(tc)
        
computeHist = [x['computeHist'] for x in flattened_results]
genAvgColor = [x['genAvgColor'] for x in flattened_results]

In [19]:
px.histogram(genAvgColor,title="Time to Compute Average")

In [20]:
px.histogram(computeHist, title="Time to Compute Hist")

In [ ]:
sum(computeHist), sum(genAvgColor)

In [ ]:
    
# def _count_tiles(slide_path, params, kwargs, position, count):
#     ts = large_image.getTileSource(slide_path)
#     lpotf = len(OutputTotals._fields)
#     total = [0] * lpotf
#     for pos in range(position, position + count):
#         tile = ts.getSingleTile(tile_position=pos, **kwargs)['tile']
#         subtotal = _count_image(tile, params)[0]
#         for k in range(lpotf):
#             total[k] += subtotal[k]
#     return OutputTotals._make(total)    